<a href="https://colab.research.google.com/github/mshumer/gpt-prompt-engineer/blob/main/gpt_prompt_engineer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Auto Prompt Engineering 
Original Notebook By Matt Shumer (https://twitter.com/mattshumer_)

Original Github repo: https://github.com/mshumer/gpt-prompt-engineer

In this notebook, we will simulate how to generate an optimal prompt/instruction for a given task.

## Installations

In [1]:
!pip install -r ../requirements.txt -qq

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sparkmagic 0.21.0 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.2.1 which is incompatible.
sphinx 7.2.6 requires docutils<0.21,>=0.18.1, but you have docutils 0.16 which is incompatible.


In [2]:
 !pip install -U langchain-community -q

## Import Libraries

In [1]:
from datasets import load_dataset, Dataset
import pandas as pd
from ast import literal_eval
import sys
from pathlib import Path
import os

In [2]:
path_to_src_code = '../'
sys.path.append(path_to_src_code)

In [3]:
from src.utils.functions import *
from src.run_autoprompt import run_autoprompts

## Import Example Dataset

We will load an example dataset from the datasets library that includes a question, ground truth, answer, and contexts. Make sure that ground truths is a list of contexts that are retrieved for the specific question. 

In [6]:
fiqa_eval = load_dataset("explodinggradients/fiqa", "ragas_eval")
data = fiqa_eval['baseline']

Generating baseline split:   0%|          | 0/30 [00:00<?, ? examples/s]

In [7]:
data.to_pandas()

,question,ground_truths,answer,contexts
0,How to deposit a cheque issued to an associate...,[Have the check reissued to the proper payee.J...,\nThe best way to deposit a cheque issued to a...,[Just have the associate sign the back and the...
1,Can I send a money order from USPS as a business?,[Sure you can. You can fill in whatever you w...,"\nYes, you can send a money order from USPS as...",[Sure you can. You can fill in whatever you w...
2,1 EIN doing business under multiple business n...,[You're confusing a lot of things here. Compan...,"\nYes, it is possible to have one EIN doing bu...",[You're confusing a lot of things here. Compan...
3,Applying for and receiving business credit,"[""I'm afraid the great myth of limited liabili...",\nApplying for and receiving business credit c...,[Set up a meeting with the bank that handles y...
4,401k Transfer After Business Closure,[You should probably consult an attorney. Howe...,\nIf your employer has closed and you need to ...,[The time horizon for your 401K/IRA is essenti...
5,What are the ins/outs of writing equipment pur...,[Most items used in business have to be deprec...,\nWriting equipment purchases off as business ...,[You would report it as business income on Sch...
6,Can a entrepreneur hire a self-employed busine...,[Yes. I can by all means start my own company ...,"\nYes, an entrepreneur can hire a self-employe...",[Yes. I can by all means start my own company ...
7,Intentions of Deductible Amount for Small Busi...,"[""If your sole proprietorship losses exceed al...",\nThe intention of deductible amounts for smal...,"[""Short answer, yes. But this is not done thro..."
8,How can I deposit a check made out to my busin...,[You should have a separate business account. ...,\nYou can deposit a check made out to your bus...,"[""I have checked with Bank of America, and the..."
9,Filing personal with 1099s versus business s-c...,[Depends whom the 1099 was issued to. If it wa...,\nFiling personal taxes with 1099s versus fili...,[Depends whom the 1099 was issued to. If it wa...


### Ragas

This section goes over how to call the Ragas library on the input dataset. This comes directly from the ragas library: https://github.com/explodinggradients/ragas

In [8]:
# add directory to aws_ragas pyproject.toml file 
!pip install ragas --quiet

In [9]:
from ragas import evaluate
from ragas.metrics import faithfulness, answer_correctness, context_precision, answer_similarity
from langchain_community.chat_models import BedrockChat
from langchain_community.embeddings import BedrockEmbeddings

First, initiate Bedrock Chat and Bedrock Embeddings (the large language models used to generate evaluation metrics from Ragas). 

In [10]:
config = {
    "region_name": "us-east-1",  # E.g. "us-east-1"
    "model_id": 'anthropic.claude-3-haiku-20240307-v1:0',  # E.g "anthropic.claude-v2"
    "model_kwargs": {"temperature": 0.4},
}
bedrock_model = BedrockChat(
    region_name=config["region_name"],
    endpoint_url=f"https://bedrock-runtime.{config['region_name']}.amazonaws.com",
    model_id=config["model_id"],
    model_kwargs=config["model_kwargs"],
)

# init the embeddings
bedrock_embeddings = BedrockEmbeddings(
    region_name=config["region_name"],
)


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `BedrockChat` was deprecated in LangChain 0.0.34 and will be removed in 0.3. An updated version of the class exists in the langchain-aws package and should be used instead. To use it run `pip install -U langchain-aws` and import as `from langchain_aws import ChatBedrock`.
  warn_deprecated(


In this example, we will generate faithfulness, answer correctness, context precision, and answer similarity metrics. 

In [11]:
score = evaluate(data,metrics=[faithfulness,answer_correctness, context_precision, answer_similarity], llm=bedrock_model,
    embeddings=bedrock_embeddings)
score.to_pandas()

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`


Evaluating:   0%|          | 0/120 [00:00<?, ?it/s]

Failed to parse output. Returning None.
Failed to parse output. Returning None.
Failed to parse output. Returning None.
Failed to parse output. Returning None.
Failed to parse output. Returning None.
Failed to parse output. Returning None.
Failed to parse output. Returning None.
Failed to parse output. Returning None.
Failed to parse output. Returning None.
Failed to parse output. Returning None.
Failed to parse output. Returning None.
Failed to parse output. Returning None.
Failed to parse output. Returning None.
Failed to parse output. Returning None.
Failed to parse output. Returning None.
Failed to parse output. Returning None.


,question,ground_truths,answer,contexts,ground_truth,faithfulness,answer_correctness,context_precision,answer_similarity
0,How to deposit a cheque issued to an associate...,[Have the check reissued to the proper payee.J...,\nThe best way to deposit a cheque issued to a...,[Just have the associate sign the back and the...,Have the check reissued to the proper payee.Ju...,NaN,0.768410,1.0,0.740307
1,Can I send a money order from USPS as a business?,[Sure you can. You can fill in whatever you w...,"\nYes, you can send a money order from USPS as...",[Sure you can. You can fill in whatever you w...,Sure you can. You can fill in whatever you wa...,NaN,0.646431,1.0,0.871439
2,1 EIN doing business under multiple business n...,[You're confusing a lot of things here. Compan...,"\nYes, it is possible to have one EIN doing bu...",[You're confusing a lot of things here. Compan...,You're confusing a lot of things here. Company...,NaN,0.302608,1.0,0.610433
3,Applying for and receiving business credit,"[""I'm afraid the great myth of limited liabili...",\nApplying for and receiving business credit c...,[Set up a meeting with the bank that handles y...,"""I'm afraid the great myth of limited liabilit...",1.000000,0.715723,1.0,0.662891
4,401k Transfer After Business Closure,[You should probably consult an attorney. Howe...,\nIf your employer has closed and you need to ...,[The time horizon for your 401K/IRA is essenti...,You should probably consult an attorney. Howev...,NaN,0.439961,1.0,0.259844
5,What are the ins/outs of writing equipment pur...,[Most items used in business have to be deprec...,\nWriting equipment purchases off as business ...,[You would report it as business income on Sch...,Most items used in business have to be depreci...,1.000000,0.624666,1.0,0.720885
6,Can a entrepreneur hire a self-employed busine...,[Yes. I can by all means start my own company ...,"\nYes, an entrepreneur can hire a self-employe...",[Yes. I can by all means start my own company ...,Yes. I can by all means start my own company a...,1.000000,0.661421,1.0,0.645683
7,Intentions of Deductible Amount for Small Busi...,"[""If your sole proprietorship losses exceed al...",\nThe intention of deductible amounts for smal...,"[""Short answer, yes. But this is not done thro...","""If your sole proprietorship losses exceed all...",1.000000,0.631389,0.0,0.425558
8,How can I deposit a check made out to my busin...,[You should have a separate business account. ...,\nYou can deposit a check made out to your bus...,"[""I have checked with Bank of America, and the...",You should have a separate business account. M...,NaN,NaN,1.0,0.759963
9,Filing personal with 1099s versus business s-c...,[Depends whom the 1099 was issued to. If it wa...,\nFiling personal taxes with 1099s versus fili...,[Depends whom the 1099 was issued to. If it wa...,Depends whom the 1099 was issued to. If it was...,NaN,0.653547,1.0,0.739187


In [13]:
score.to_pandas().to_csv("../data/ragas_output.csv", index=False)

## Automatic Prompt Generation

Here, automatic prompt generation is done using specific templates (available in instruction_generation_templates folder). Feel free to add more txt files to the list below based on the name of the templates in that folder. 

In [4]:
files = [
    "one-paragraph_instruction_with_demo.txt", 
    "one-sentence_instruction_with_demo.txt", 
    "step-by-step_instruction_with_demo.txt"
]

In [5]:
# use the csv file uploaded from Ragas as the input to autoprompt. 
run_autoprompts(True, files, path_to_ragas_outputs="../data/ragas_output.csv")

/home/ec2-user/SageMaker/autoPromptEng/notebooks/../src/run_autoprompt.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fix_prompts['contexts'] = fix_prompts['contexts'].apply(literal_eval)
/home/ec2-user/SageMaker/autoPromptEng/notebooks/../src/run_autoprompt.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fix_prompts['ground_truths'] = fix_prompts['ground_truths'].apply(
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecation

Error parsing JSON: Expecting property name enclosed in double quotes: line 2 column 4 (char 5)
Error parsing JSON: Expecting property name enclosed in double quotes: line 2 column 4 (char 5)
9


  0%|                                          | 0/27 [00:00<?, ?it/s]

Gen 1:   Here is a one paragraph instruction on how to solve the task without mentioning any of the content from the context:

To address the question being asked, carefully consider each component of the scenario described and think through the logical steps required to derive the answer from the information and details provided without directly restat
Gen 2:   Here is a one paragraph instruction on how to approach the task without mentioning any specific details:

To successfully complete the task at hand, one should carefully analyze what the final goal or desired outcome is. With the end result in mind, work backwards to identify the key steps or components required to achieve
----------


  7%|██▌                               | 2/27 [00:04<00:56,  2.25s/it]

...

Draw
Gen 1:   Here is a one sentence instruction for the task without referencing any context:

Examine the available information and details provided, analyze what is being asked, identify the necessary steps or process required, and formulate a concise response expressing how to complete what is requested.
Gen 2:   Here is a one sentence instruction for how to solve the task without mentioning any content from the provided context: Think conceptually and indirectly answer the question by analyzing the implications of the context's substance rather than reproducing excerpts.
----------


100%|█████████████████████████████████| 27/27 [01:44<00:00,  4.52s/it]

Draw
Gen 1:   Here is a step-by-step instruction for the task without mentioning any context content:

1. Clearly identify the overall goal or objective of the task. Break it down into specific, well-defined steps if needed.

2. Evaluate what information or resources you will require to complete
Gen 2:   Here is a step-by-step instruction for the task without mentioning any content from the provided context:

1. Identify the key elements of the task based on the question asked. Determine what information or outcomes are explicitly requested in the question.

2. Analyze the broader context
----------


100%|█████████████████████████████████| 27/27 [01:48<00:00,  4.02s/it]

Draw


The output to running the autoprompt function above is stored in the data folder of this repository as 'new_prompts.csv'. In addition, a list of prompts are outputted in the data folder as 'prompt_id.csv'. Below is the output of the best three instructions for each question in the dataset. 

In [6]:
new_prompts = pd.read_csv('../data/new_prompts.csv')

In [7]:
new_prompts.head()

,question,ground_truths,answer,contexts,ground_truth,faithfulness,answer_correctness,context_precision,answer_similarity,best_prompt,second_best_prompt,third_best_prompt
0,1 EIN doing business under multiple business n...,"[""You're confusing a lot of things here. Compa...","\nYes, it is possible to have one EIN doing bu...",['You\'re confusing a lot of things here. Comp...,You're confusing a lot of things here. Company...,NaN,0.302608,1.0,0.610433,Give careful consideration to the question and...,"Consider the question and provided context, th...",Read the provided context and question careful...
1,401k Transfer After Business Closure,"[""You should probably consult an attorney. How...",\nIf your employer has closed and you need to ...,['The time horizon for your 401K/IRA is essent...,You should probably consult an attorney. Howev...,NaN,0.439961,1.0,0.259844,Study the question and background to grasp the...,Consider the question and situation holistical...,Contemplate the question and context as a whol...
2,Intentions of Deductible Amount for Small Busi...,"['""If your sole proprietorship losses exceed a...",\nThe intention of deductible amounts for smal...,"['""Short answer, yes. But this is not done thr...","""If your sole proprietorship losses exceed all...",1.0,0.631389,0.0,0.425558,"Consider the question prompt, provided context...","Study the question, context and desired output...","Examine the question, context and desired outp..."
3,"How can I make $250,000.00 from trading/invest...","['Deposit $3,500 each month in a brokerage acc...","\nMaking $250,000.00 from trading/investing/bu...","['""The answer to your question is Forex tradin...","Deposit $3,500 each month in a brokerage accou...",NaN,0.445880,0.0,0.724696,Read thoroughly the question and background pr...,Analyze the question and context to understand...,"Examine the question and provided information,..."
4,What approaches are there for pricing a small ...,"[""I don't have any experience in this, but thi...",\nThere are several approaches for pricing a s...,"[""I don't have any experience in this, but thi...","I don't have any experience in this, but this ...",1.0,0.184233,1.0,0.736934,Thoughtfully consider the question and context...,Carefully review the given inputs to gain a fu...,"To thoughtfully address this task, first ensur..."


Below are the prompts/instructions mapped to ids. 

In [8]:
prompt_ids = pd.read_csv('../data/prompt_ids.csv')

In [9]:
prompt_ids.head()

,Id,Prompts
0,0,Give careful consideration to the question and...
1,1,Study the question and background to grasp the...
2,2,"Consider the question prompt, provided context..."
3,3,Read thoroughly the question and background pr...
4,4,Thoughtfully consider the question and context...
